# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,buka,40.8108,69.1986,2.32,82,100,3.22,UZ,1705274486
1,1,nelson bay,-32.7167,152.1500,24.49,80,75,8.05,AU,1705274486
2,2,albany,42.6001,-73.9662,-3.92,58,39,0.89,US,1705274192
3,3,bilibino,68.0546,166.4372,-33.72,100,99,1.42,RU,1705274486
4,4,edinburgh of the seven seas,-37.0676,-12.3116,15.66,88,98,3.32,SH,1705274487


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
city_humidity_plot = city_data_df.hvplot.points(
    x='Lng', 
    y='Lat', 
    geo=True, 
    tiles='OSM',
    hover_cols=['City', 'Country'], 
    color='Humidity', 
    cmap='viridis',
    size='Humidity', 
    scale=0.5,
    title='City Humidity'
)

# Display the map
city_humidity_plot.opts(width=800, height=600)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Define criteria for ideal weather conditions
low_temp = 24
high_temp = 40
high_hum = 85
low_hum = 45
low_wind_speed = 6
high_wind_speed = 20

# Narrow down cities based on weather conditions
ideal_weather_df = city_data_df[
    (city_data_df['Max Temp'] > low_temp) & 
    (city_data_df['Max Temp'] < high_temp) & 
    (city_data_df['Wind Speed'] > low_wind_speed) & 
    (city_data_df['Wind Speed'] < high_wind_speed) & 
    (city_data_df['Humidity'] > low_hum) & 
    (city_data_df['Humidity'] < high_hum)
]


# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display the DataFrame with ideal weather conditions
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,nelson bay,-32.7167,152.1500,24.49,80,75,8.05,AU,1705274486
9,9,uturoa,-16.7333,-151.4333,27.72,80,96,10.78,PF,1705274488
156,156,andergrove,-21.0833,149.1833,28.14,75,87,6.41,AU,1705274514
159,159,port mathurin,-19.6833,63.4167,28.18,76,100,6.04,MU,1705274515
185,185,dhuusamarreeb,5.5350,46.3861,24.45,74,79,6.33,SO,1705274517


### Step 3: Create a new DataFrame called `hotel_df`.

In [23]:
# Create hotel_df DataFrame with selected columns
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
1,nelson bay,AU,-32.7167,152.1500,80,
9,uturoa,PF,-16.7333,-151.4333,80,
156,andergrove,AU,-21.0833,149.1833,75,
159,port mathurin,MU,-19.6833,63.4167,76,
185,dhuusamarreeb,SO,5.5350,46.3861,74,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [24]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "type": "accommodation.hotel",
    "apiKey": geoapify_key, 
    "radius": radius,
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    lat = row['Lat']
    lng = row['Lng']
    
    # Set base URL for API request
    base_url = f"https://api.geoapify.com/v2/places?&lat={lat}&lon={lng}"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    hotel_info = response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
nelson bay - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
andergrove - nearest hotel: No hotel found
port mathurin - nearest hotel: Escale Vacances
dhuusamarreeb - nearest hotel: centaral hotel
cam ranh - nearest hotel: Khách sạn Thái Bình
callao - nearest hotel: Casa Ronald
mili - nearest hotel: Marshall Islands Resort
utrik - nearest hotel: No hotel found
sarangani - nearest hotel: Lemlunay Resort
inarajan village - nearest hotel: Leo Palace
avarua - nearest hotel: Paradise Inn
enewetak - nearest hotel: No hotel found
betio village - nearest hotel: Seaman Hostel
rongelap - nearest hotel: No hotel found
saipan - nearest hotel: Chalan Kanoa Beach Hotel
mejit - nearest hotel: Robert Reimers Enterprises
weno - nearest hotel: High Tide Hotel
kingston - nearest hotel: Aphrodite Hotel
afaahiti - nearest hotel: Omati Lodge
lubang - nearest hotel: Matabungkay Beach Hotel
wajir - nearest hotel: Kilimanjaro
acarau - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
1,nelson bay,AU,-32.7167,152.1500,80,No hotel found
9,uturoa,PF,-16.7333,-151.4333,80,Hawaiki Nui hotel
156,andergrove,AU,-21.0833,149.1833,75,No hotel found
159,port mathurin,MU,-19.6833,63.4167,76,Escale Vacances
185,dhuusamarreeb,SO,5.5350,46.3861,74,centaral hotel
196,cam ranh,VN,11.9214,109.1591,69,Khách sạn Thái Bình
202,callao,PE,-12.0667,-77.1500,73,Casa Ronald
239,mili,MH,6.0815,171.7350,73,Marshall Islands Resort
258,utrik,MH,11.2278,169.8474,73,No hotel found
266,sarangani,PH,5.4033,125.4636,77,Lemlunay Resort


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [25]:
%%capture --no-display

# Configure the map plot with additional hover information
map_plot = hotel_df.hvplot.points(
    'Lng', 'Lat', 
    geo=True,
    tiles='OSM',
    hover_cols=['City', 'Country', 'Hotel Name'],  # Add 'Hotel Name' to hover information
    size='Humidity',  # Setting size based on humidity
    color='Humidity',
    colormap='viridis',
    height=500,
    width=800,
    title='City Hotels with Humidity'
)

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)